## Importing all required libraries

In [3]:
import os
import requests
import json
cwd = os.getcwd()

In [4]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib as mpl
import seaborn as sb
import numpy as np
import pandas as pd
from pandas import Series
import zipfile

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
rcParams['figure.figsize'] = 5, 4
sb.set_style('whitegrid')


pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# Gathering Data

## Getting the Twitter Archive

In [5]:
df = pd.read_csv('twitter-archive-enhanced.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [6]:
df_append = df.copy()

In [7]:
df_append.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


## Getting the extra data from Twitter API

In [8]:
import tweepy

consumer_key = 'ejjJsGZP91PETUTLFQnl1I47g'
consumer_secret = 'tT2dnqoWcJonM1RGCVkMl6net7kBYRsUsHnD12r5MpAI35Zdwv'
access_token = '49008389-54oYNUfoyb0GGuaKRSWlhZQKn19RQq9AI5ZwqaRdA'
access_secret = 'K4qvZSLINbo8tnF7e3AJtEQR8tn3GVHuIX3joiUfptrbV'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

### Writing a JSON file storing all the tweets

In [9]:
tweet = []
tweet_dict = {"tweet_id" :[], "tweet_status" :[],
              "full_text" :[], "retweet_count": [], "favorite_count" : []}
for ids in enumerate(df.tweet_id):
    try:
        status = api.get_status(ids[1], tweet_mode='extended')
        
        tweet.append(status)
        tweet_dict["tweet_status"].append(status)
        tweet_dict["tweet_id"].append(status.id)
        tweet_dict["full_text"].append(status.full_text)
        tweet_dict["retweet_count"].append(status.retweet_count)
        tweet_dict["favorite_count"].append(status.favorite_count)
    except:
        pass

Rate limit reached. Sleeping for: 596
Rate limit reached. Sleeping for: 594


### converting the created doctionary into a Pandas DataFrame

In [10]:
tweet_df = pd.DataFrame.from_dict(tweet_dict )
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 5 columns):
favorite_count    2345 non-null int64
full_text         2345 non-null object
retweet_count     2345 non-null int64
tweet_id          2345 non-null int64
tweet_status      2345 non-null object
dtypes: int64(3), object(2)
memory usage: 91.7+ KB


In [11]:
tweet_df.head()

,favorite_count,full_text,retweet_count,tweet_id,tweet_status
0,38994,This is Phineas. He's a mystical boy. Only eve...,8653,892420643555336193,"Status(truncated=False, _api=<tweepy.api.API o..."
1,33390,This is Tilly. She's just checking pup on you....,6355,892177421306343426,"Status(truncated=False, _api=<tweepy.api.API o..."
2,25168,This is Archie. He is a rare Norwegian Pouncin...,4220,891815181378084864,"Status(truncated=False, _api=<tweepy.api.API o..."
3,42347,This is Darla. She commenced a snooze mid meal...,8768,891689557279858688,"Status(truncated=False, _api=<tweepy.api.API o..."
4,40514,This is Franklin. He would like you to stop ca...,9531,891327558926688256,"Status(truncated=False, _api=<tweepy.api.API o..."


#### Storing the Data Frame into a CSV file

In [12]:
tweet_df.to_csv('tweet_df.csv')

### Writing the Tweet Statuses into a txt file:

In [13]:
pd.DataFrame(tweet).to_csv(r'tweet_json.txt', header=None, index=None, sep=' ', mode='a')

### Reading the JSON data from the "tweet_json.txt" into a DataFrame

In [7]:
json_df = pd.read_csv('tweet_json.txt', header = None)

## Downloading the TSV file Programmatically

In [130]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

In [131]:
response = requests.get(url)
with open("image_predictions.tsv", mode='wb') as file:
        file.write(response.content)

### Reading the TSV file into a DataFrame

In [132]:
df_immage = pd.read_csv('image_predictions.tsv', sep = '\t')
df_immage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB
